In [0]:
from pyspark.sql.types import *
from datetime import datetime

# Define schema explicitly
transaction_type_schema = StructType([
    StructField("TYPE_CODE", StringType(), False),
    StructField("TYPE_NAME", StringType(), False),
    StructField("DESCRIPTION", StringType(), True),
    StructField("IS_FINANCIAL", BooleanType(), False),
    StructField("IS_REVERSAL_TYPE", BooleanType(), False),
    StructField("CREATED_DATE", TimestampType(), False),
    StructField("LAST_UPDATED_DATE", TimestampType(), False)
])

# Sample static data
data = [
    ('001', 'Cash Withdrawal', 'Customer withdraws cash from their account.', True, False, datetime.now(), datetime.now()),
    ('002', 'Cash Deposit', 'Customer deposits cash into their account.', True, False, datetime.now(), datetime.now()),
    ('003', 'Funds Transfer (Internal)', 'Transfer of funds between accounts within the same bank.', True, False, datetime.now(), datetime.now()),
    ('004', 'Bill Payment', 'Customer pays a bill (e.g., utility, credit card).', True, False, datetime.now(), datetime.now()),
    ('005', 'Cheque Deposit', 'Customer deposits a cheque into their account.', True, False, datetime.now(), datetime.now()),
    ('010', 'ATM Replenishment', 'Bank staff loads cash into an ATM machine.', True, False, datetime.now(), datetime.now()),
    ('011', 'ATM Cash Collection', 'Bank staff collects excess cash from an ATM machine.', True, False, datetime.now(), datetime.now()),
    ('020', 'Service Fee', 'General service charge applied to an account.', True, False, datetime.now(), datetime.now()),
    ('021', 'ATM Withdrawal Fee (Other Bank)', 'Fee charged for withdrawing cash from another bank\'s ATM.', True, False, datetime.now(), datetime.now()),
    ('022', 'Interbank Transfer Fee', 'Fee for transferring funds to another bank.', True, False, datetime.now(), datetime.now()),
    ('030', 'Loan Disbursement', 'Bank disburses funds for a loan.', True, False, datetime.now(), datetime.now()),
    ('031', 'Loan Repayment', 'Customer makes a payment towards a loan.', True, False, datetime.now(), datetime.now()),
    ('032', 'Credit Card Payment', 'Customer pays their credit card bill.', True, False, datetime.now(), datetime.now()),
    ('040', 'FX Buy (Bank Buys)', 'Bank buys foreign currency from a customer.', True, False, datetime.now(), datetime.now()),
    ('041', 'FX Sell (Bank Sells)', 'Bank sells foreign currency to a customer.', True, False, datetime.now(), datetime.now()),
    ('901', 'Withdrawal Reversal', 'Reversal of a previously processed cash withdrawal.', True, True, datetime.now(), datetime.now()),
    ('902', 'Deposit Reversal', 'Reversal of a previously processed cash deposit.', True, True, datetime.now(), datetime.now()),
    ('903', 'Transfer Reversal', 'Reversal of a previously processed funds transfer.', True, True, datetime.now(), datetime.now()),
    ('904', 'Fee Reversal', 'Reversal of a previously charged fee.', True, True, datetime.now(), datetime.now()),
    ('999', 'General Adjustment', 'General accounting adjustment (may or may not be a reversal of a specific transaction).', True, False, datetime.now(), datetime.now())
]

# Create DataFrame
ref_transaction_types_df = spark.createDataFrame(data, schema=transaction_type_schema)

# Define path
#dimension_path = "/mnt/bank_cbs/bronze/dimensions/ref_transaction_types"

# Save as Delta
ref_transaction_types_df.write.mode("overwrite").format("delta").saveAsTable("bank_cbs.bronze.ref_transaction_types")

In [0]:
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, TimestampType

# 1. Initialize Spark (if not already)
spark = SparkSession.builder.getOrCreate()

# 2. Prepare your data
now = datetime.now()

currencies_data = [
    ("BDT", "Bangladeshi Taka", 2, "৳", True, now, now),
    ("USD", "United States Dollar", 2, "$", True, now, now),
    ("EUR", "Euro", 2, "€", True, now, now),
    ("GBP", "British Pound", 2, "£", True, now, now),
    ("JPY", "Japanese Yen", 0, "¥", True, now, now),
    ("INR", "Indian Rupee", 2, "₹", True, now, now),
    ("CNY", "Chinese Yuan", 2, "¥", True, now, now),
    ("SAR", "Saudi Riyal", 2, "﷼", True, now, now),
]

# 3. Define the schema
currencies_schema = StructType([
    StructField("currency_code", StringType(), False),
    StructField("currency_name", StringType(), False),
    StructField("decimal_places", IntegerType(), False),
    StructField("symbol", StringType(), True),
    StructField("is_active", BooleanType(), False),
    StructField("created_date", TimestampType(), False),
    StructField("updated_date", TimestampType(), False),
])

# 4. Create DataFrame
ref_currencies_df = spark.createDataFrame(data=currencies_data, schema=currencies_schema)


# 5. Write to Unity Catalog volume (Bronze zone)
ref_currencies_df.write.format("delta").mode("overwrite").saveAsTable("bank_cbs.bronze.ref_currencies")



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, TimestampType
from datetime import datetime

# Initialize Spark Session
spark = SparkSession.builder.getOrCreate()

now = datetime.now()

# Data
channels_data = [
    ("100", "ATM", "Transactions initiated via Automated Teller Machines.", True, now, now),
    ("101", "Mobile Banking App", "Transactions initiated via the bank's mobile application.", True, now, now),
    ("102", "Internet Banking", "Transactions initiated via the bank's web portal.", True, now, now),
    ("103", "Bank Branch", "Transactions initiated in person at a physical bank branch.", True, now, now),
    ("104", "POS Terminal", "Point of Sale transactions (e.g., merchant payments).", True, now, now),
    ("105", "Call Center", "Transactions initiated through the bank's customer call center.", True, now, now),
    ("106", "API Integration", "Transactions initiated programmatically via direct API integrations (e.g., corporate clients, fintech partners).", True, now, now),
    ("107", "Kiosk", "Self-service transactions initiated via bank kiosks.", True, now, now),
    ("108", "Agent Banking", "Transactions facilitated by authorized agent banking outlets.", True, now, now),
]

# Schema
channels_schema = StructType([
    StructField("channel_code", StringType(), False),
    StructField("channel_name", StringType(), False),
    StructField("description", StringType(), True),
    StructField("is_active", BooleanType(), False),
    StructField("created_date", TimestampType(), False),
    StructField("last_updated_date", TimestampType(), False),
])

#create Dataframe
ref_channels_df = spark.createDataFrame(data=channels_data, schema=channels_schema)


# Write to Unity Catalog volume (Bronze zone)
ref_channels_df.write.format("delta").mode("overwrite").saveAsTable("bank_cbs.bronze.ref_channels")

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from datetime import datetime

#now = datetime.now()

# Define data
data = [
    ("200", "Core Banking System", "The central system managing customer accounts, balances, and core transactions.", datetime.now(), datetime.now()),
    ("201", "ATM Gateway", "Handles communication and processing for Automated Teller Machines.", datetime.now(), datetime.now()),
    ("202", "Mobile Banking Platform", "Manages transactions originating from the mobile banking application.", datetime.now(), datetime.now()),
    ("203", "Internet Banking Platform", "Manages transactions originating from the internet banking web portal.", datetime.now(), datetime.now()),
    ("204", "Loan Management System", "Manages the lifecycle of loan products, from origination to repayment.", datetime.now(), datetime.now()),
    ("205", "Foreign Exchange Engine", "Processes currency exchange transactions and calculates rates.", datetime.now(), datetime.now()),
    ("206", "Fee Calculation Engine", "Automates the calculation and application of various bank fees.", datetime.now(), datetime.now()),
    ("207", "Clearing House Integration", "Manages communication and settlement with external clearing houses (e.g., for interbank transfers).", datetime.now(), datetime.now()),
    ("208", "Fraud Detection System", "Monitors transactions for suspicious activity and flags potential fraud.", datetime.now(), datetime.now()),
    ("209", "Cash Management System", "Manages physical cash flows, including ATM replenishment and vault management.", datetime.now(), datetime.now()),
    ("210", "Payment Gateway", "Integrates with external payment networks for merchant and bill payments.", datetime.now(), datetime.now()),
    ("211", "General Ledger System", "The system responsible for maintaining the bank's official accounting records and financial statements.", datetime.now(), datetime.now())
]

# Define schema
schema = StructType([
    StructField("SYSTEM_CODE", StringType(), False),
    StructField("SYSTEM_NAME", StringType(), False),
    StructField("DESCRIPTION", StringType(), True),
    StructField("CREATED_DATE", TimestampType(), False),
    StructField("LAST_UPDATED_DATE", TimestampType(), False),
])

# Create DataFrame
ref_processing_systems_df = spark.createDataFrame(data=data, schema=schema)

# Write to Unity Catalog volume (Bronze zone)
ref_processing_systems_df.write.format("delta").mode("overwrite").saveAsTable("bank_cbs.bronze.ref_processing_systems")



In [0]:
# Data for REF_TRANSACTION_STATUS
transaction_status_data = [
    ("300", "Pending", "Transaction initiated, awaiting initial processing or authorization."),
    ("301", "Authorized", "Funds reserved for the transaction, but not yet fully processed or settled."),
    ("302", "Processing", "Transaction is actively being processed by internal or external systems."),
    ("303", "Success", "Transaction completed successfully, and all financial impacts have been recorded."),
    ("304", "Failed", "Transaction terminated due to an error; no financial impact or impact was rolled back."),
    ("305", "Reversed", "A previously successful transaction has been financially negated by offsetting entries."),
    ("306", "Cancelled", "Transaction was stopped by the user or system before completion, typically with no financial impact."),
    ("307", "Settled", "Funds have been successfully cleared and posted, indicating final completion for interbank or batch processes."),
    ("308", "Declined", "Transaction was rejected at an early stage, typically by an external system or for policy reasons (e.g., insufficient funds)."),
    ("309", "Refunded", "Funds were returned to the customer after a successful transaction (a new, separate transaction).")
]

# Schema for REF_TRANSACTION_STATUS
transaction_status_schema = StructType([
    StructField("status_code", StringType(), False),
    StructField("status_name", StringType(), False),
    StructField("description", StringType(), True)
])

# Create DataFrame
ref_transaction_status_df = spark.createDataFrame(data=transaction_status_data, schema=transaction_status_schema)

# Save as Delta Table
ref_transaction_status_df.write.format("delta").mode("overwrite").saveAsTable("bank_cbs.bronze.ref_transaction_status")

In [0]:
# Define data for gl_accounts
now = datetime.now()

gl_accounts_data = [
    ("1000", "Assets", "ASSET", None, True, now, now),
    ("2000", "Liabilities", "LIABILITY", None, True, now, now),
    ("3000", "Equity", "EQUITY", None, True, now, now),
    ("4000", "Revenue", "REVENUE", None, True, now, now),
    ("5000", "Expenses", "EXPENSE", None, True, now, now),
    ("1001", "Cash and Cash Equivalents", "ASSET", "1000", True, now, now),
    ("100101", "Bank Vault Cash", "ASSET", "1001", True, now, now),
    ("100102", "Cash in ATM Machines", "ASSET", "1001", True, now, now),
    ("1010", "Customer Deposit Accounts (Asset Side)", "ASSET", "1000", True, now, now),
    ("101001", "Customer Savings Accounts", "ASSET", "1010", True, now, now),
    ("101002", "Customer Current Accounts", "ASSET", "1010", True, now, now),
    ("1020", "Loans Receivable", "ASSET", "1000", True, now, now),
    ("1030", "Interbank Accounts Receivable", "ASSET", "1000", True, now, now),
    ("1040", "Fixed Assets", "ASSET", "1000", True, now, now),
    ("2010", "Customer Deposits (Liability Side)", "LIABILITY", "2000", True, now, now),
    ("201001", "Customer Savings Deposits", "LIABILITY", "2010", True, now, now),
    ("201002", "Customer Current Deposits", "LIABILITY", "2010", True, now, now),
    ("2020", "Borrowed Funds", "LIABILITY", "2000", True, now, now),
    ("2030", "Interbank Accounts Payable", "LIABILITY", "2000", True, now, now),
    ("2040", "Accrued Expenses", "LIABILITY", "2000", True, now, now),
    ("3010", "Share Capital", "EQUITY", "3000", True, now, now),
    ("3020", "Retained Earnings", "EQUITY", "3000", True, now, now),
    ("4010", "Interest Income", "REVENUE", "4000", True, now, now),
    ("4020", "Fee and Commission Income", "REVENUE", "4000", True, now, now),
    ("402001", "ATM Service Fee Income", "REVENUE", "4020", True, now, now),
    ("402002", "Interbank Fee Income", "REVENUE", "4020", True, now, now),
    ("4030", "Foreign Exchange Gain/Loss", "REVENUE", "4000", True, now, now),
    ("5010", "Interest Expense", "EXPENSE", "5000", True, now, now),
    ("5020", "Operating Expenses", "EXPENSE", "5000", True, now, now),
    ("502001", "ATM Operating Expenses", "EXPENSE", "5020", True, now, now),
    ("5030", "Provision for Loan Losses", "EXPENSE", "5000", True, now, now)
]


# Define schema for gl_accounts
gl_accounts_schema = StructType([
    StructField("GL_CODE", StringType(), False),
    StructField("GL_NAME", StringType(), False),
    StructField("ACCOUNT_TYPE", StringType(), False),
    StructField("PARENT_GL_CODE", StringType(), True),
    StructField("IS_ACTIVE", BooleanType(), False),
    StructField("CREATED_DATE", TimestampType(), False),
    StructField("LAST_UPDATED_DATE", TimestampType(), False)
])

# Create DataFrame
ref_gl_accounts_df = spark.createDataFrame(data=gl_accounts_data, schema=gl_accounts_schema)

# Save as Delta Table
ref_gl_accounts_df.write.format("delta").mode("overwrite").saveAsTable("bank_cbs.bronze.ref_gl_accounts")